# Fall 2022 CE170A HW2: Sensor Data Analysis
v3 - Fall 2022: Prof. Kenichi Soga, Jaewon Saw, Shih-Hung Chiu

v2 - Fall 2021: Prof. Kenichi Soga, Renjie Wu, Yaobin Yang, James Wang 

In [ ]:
import os
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2021/master/Homeworks/HW2/data/Fall2022_data.zip" -O Fall2022_data.zip
!unzip Fall2022_data

## Background

There are two ways to understand phenomena scientifically. One is through mathematical modeling and computer simulation, which is the purpose of our quiz 2 (truss finite element simulation). The other is through experiments, which are performed in this class (model bridge loading experiment). By comparing experimental results with simulation results, one may appreciate the complexity of real-world phenomena and have a deeper understanding of the power and limitations of the simulation method.




<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/overview.png" alt="Node numbering" width="600" />
    
    Figure 1. Simulation Vs. Experiment
</center>

In this assignment, you are going to analyze the sensor data from the model bridge experiment and compare the result with the truss simulation performed in Quiz 2. The data you are going to explore are as follows:
* Displacement data from displacement sensors (potentiometric measuring devices),
* Tilt data from WSN tilt motes, and 
* Distributed (spatial) strain data from fiber optics sensors.


The typical workflow of data analysis is summarized in Figure 2. 
<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/data_processing.png" alt="Node numbering" width="600" />
    
    Figure 2. Typical workflow of data analysis
</center>

We start with the raw data collected from the sensor systems. Then, we preprocess the raw data and store the cleaned data into a data storage system. There are various reasons for data preprocessing. Missing data handling, experiment/equipment setting adjustment, and data format conversion are common reasons (there are more, of course). 

Most data can be represented as a single spreadsheet. Such data include the displacement data and tilt data that we are going to explore in this assignment. However, in some cases, data can have complex structures; such as the distributed strain data we have in this case. The distributed strain data is collected from fiber optics sensors, which record the strain along with the fiber (spatial component) during the experiment (time component). As you all know, we deployed numerous fiber optics sensors (with different lengths) across different structure members to monitor the load responses of the model bridge. Since we have three model bridges this year, each with different sensor deployment strategies, the sensors' raw data are huge and hard to handle. For the ease of this exercise, the distributed strain data is preprocessed by us, and only some cleaned data from certain bridge members are presented here.


Having the cleaned data on our hands, we can perform various postprocessing techniques such as statistical analysis, and most importantly, visualize the data for interpretation.

The data are given in a format that follows the numbering given in the following figures.

Figure 3 shows the nodal numbering of the model bridge.
<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/bridge_diagram.png" alt="Node numbering" width="500" />
    
    Figure 3. Node numbering for the model bridge
</center>


## Videos from bridge tests
Refer to the videos <a href="https://drive.google.com/drive/folders/1HdlADyA2HnBFveScQDFVCDR5zo17PDfv?usp=sharing">here</a>
 to look back on any details you need for the following sections.

# Data Exploration

We tested three bridges: (1) Single-angle bridge, (2) Double-angle bridge, (3) Double-angle bridge with defect. Throughout this assignment, we will use the indices 1, 2, and 3 to refer to those bridges. 

Let's first install the required libraries and then read the processed data for the three bridges (in desired units and format) that we'll need for our analysis.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# (1) Loading processes
Let's take a look at the loading processes first.

In [ ]:
lbs_to_kN = 4.4482216153/1000

load1 = np.loadtxt("load1.txt") * lbs_to_kN
load2 = np.loadtxt("load2.txt") * lbs_to_kN
load3 = np.loadtxt("load3.txt") * lbs_to_kN

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.plot(load1, label='1: Single-Angle')
ax.plot(load2, label='2: Double-Angle')
ax.plot(load3, label='3: Double-Angle with Defect')
plt.ylabel('Axial force (kN)', fontsize = 10)
plt.xlabel('Time Steps', fontsize = 10)
plt.ylim([0, 75])

ax.legend()
ax.grid(color = [0.8, 0.8, 0.8], which = 'major')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.tick_params(direction="in", which = 'major', length = 6, width = 1)
ax.tick_params(direction="in", which = 'minor', length = 3, width = 0.6)
ax.set_title('Load vs. Time', fontsize = 14)

<font color='red'>Question: </font> For each bridge, what were the maximum load and the corresponding number of time steps taken to reach the maximum load? Does the above plot match your on-site observations? Is the plot as you would expect?

<font color='red'>Answer: </font>

# (2) Expansion of bottom members

Now let's take a look at how the bottom members responded to the load. To see the expansion of those members, we will need data from the linear variable differential transformers (LVDTs) that we placed at each end of the bottom members. As seen from Figures 3 and 4, we see that we need LVDTs 2 and 7 for member AB and LVDTs 3 and 6 for member DC. 
<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/lvdt_position.jpg" alt="Node numbering" width="800" />
    
    Figure 4.1 LVDT configuration for the experiment. 
</center>

<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/figure4.png" alt="A bridge." width="600" />
    
    Figure 4.2 Dimensions of the model
</center>

Let's plot the load vs. displacement curves for LVDTs 2, 3, 6, and 7, using the plotting function below. Note that the plot_lines function is capable of plotting multiple lines.

In [ ]:
def plot_lines(xs,ys,xy_labels,line_lables, title_name, xlim_end):
    """ A multi-line plotter
     Args:
      xs: list of x values, i.e. [x1,x2,x3...]
      ys: list of y values, i.e. [y1,y2,y3...]
      xy_labels: labels for the figure, in the form of (x_label, y_label)
      line_lables: labels for plotted lines, must match to the order of yvalues in y

    Returns:
      ax: matplotlib axes 
    """
    assert len(xs)==len(ys), "number of x must equal to number of y"
    assert len(ys)==len(line_lables),  "ys and line_lables must be list with same shape, i.e. [y1],[y1_label]"
    
    fig = plt.figure()
    ax=fig.add_axes([0,0,1,1])
    ax.grid(color = [0.8, 0.8, 0.8], which = 'major')
    
    for x,y in zip(xs,ys):
        l = ax.plot(x,y)
    ax.set_xlabel(xy_labels[0])
    ax.set_ylabel(xy_labels[1])
    ax.legend(labels = line_lables)
    
    ax.set_title(title_name,
             fontsize = 14)
    
    ax.set_xlim([0, xlim_end])
    
    return ax

## (2.1) Member AB

We first load the LVDT data for member AB:

In [ ]:
inch_to_m = 0.0254

LVDT2_bridge1 = np.loadtxt("LVDT2_bridge1.txt") * inch_to_m
LVDT7_bridge1 = np.loadtxt("LVDT7_bridge1.txt") * inch_to_m

LVDT2_bridge2 = np.loadtxt("LVDT2_bridge2.txt") * inch_to_m
LVDT7_bridge2 = np.loadtxt("LVDT7_bridge2.txt") * inch_to_m

LVDT2_bridge3 = np.loadtxt("LVDT2_bridge3.txt") * inch_to_m
LVDT7_bridge3 = np.loadtxt("LVDT7_bridge3.txt") * inch_to_m

Plot the individual LVDT 2 and LVDT 7 data for the three bridges:

In [ ]:
# The following indices (ind1, ind2, ind3) are used for plotting the LVDT data up to 
# approximately their maximum magnitudes. Feel free to change these indices and see what the plot looks like!

ind1 = 2133
ind2 = 1645
ind3 = 740

plot_lines([load1[0:ind1], load2[0:ind2], load3[0:ind3]],[LVDT2_bridge1[0:ind1], LVDT2_bridge2[0:ind2], LVDT2_bridge3[0:ind3]],
           ['Load (kN)','Displacement (m)'],['1: Single-Angle','2: Double-Angle', '3: Double-Angle with Defect'], 'LVDT 2', 60)

In [ ]:
plot_lines([load1[0:ind1], load2[0:ind2], load3[0:ind3]],[LVDT7_bridge1[0:ind1], LVDT7_bridge2[0:ind2], LVDT7_bridge3[0:ind3]],
           ['Load (kN)','Displacement (m)'],['1: Single-Angle','2: Double-Angle', '3: Double-Angle with Defect'], 'LVDT 7', 60)

Now, we use the LVDT 2 and LVDT 7 data to plot the expansion of member AB:

In [ ]:
plot_lines([load1[0:ind1], load2[0:ind2], load3[0:ind3]],[LVDT2_bridge1[0:ind1]+LVDT7_bridge1[0:ind1], LVDT2_bridge2[0:ind2]+LVDT7_bridge2[0:ind2], LVDT2_bridge3[0:ind3]+LVDT7_bridge3[0:ind3]],
           ['Load (kN)','Displacement (m)'],['1: Single-Angle','2: Double-Angle', '3: Double-Angle with Defect'], 'Member AB Expansion', 60)

<font color='red'>Question: </font> For each of the three bridges, describe the behavior of the LVDTs and the behavior of the bottom member using the plotted figures above. Given these displacements, what strains would you expect in the bottom members? How do the experimental results compare to your Quiz 2 calculations? 

<font color='red'>Answer:</font> 

## (2.2) Member DC
We repeat the process for member DC with LVDTs 3 and 6:

In [ ]:
LVDT3_bridge1 = np.loadtxt("LVDT3_bridge1.txt") * inch_to_m
LVDT6_bridge1 = np.loadtxt("LVDT6_bridge1.txt") * inch_to_m

LVDT3_bridge2 = np.loadtxt("LVDT3_bridge2.txt") * inch_to_m
LVDT6_bridge2 = np.loadtxt("LVDT6_bridge2.txt") * inch_to_m

LVDT3_bridge3 = np.loadtxt("LVDT3_bridge3.txt") * inch_to_m
LVDT6_bridge3 = np.loadtxt("LVDT6_bridge3.txt") * inch_to_m

In [ ]:
# The following indices (ind1, ind2, ind3) are used for plotting the LVDT data up to 
# approximately their maximum magnitudes. Feel free to change these indices and see what the plot looks like!

ind1 = 2133
ind2 = 1660
ind3 = 640

plot_lines([load1[0:ind1], load2[0:ind2], load3[0:ind3]],[LVDT3_bridge1[0:ind1], LVDT3_bridge2[0:ind2], LVDT3_bridge3[0:ind3]],
           ['Load (kN)','Displacement (m)'],['1: Single-Angle','2: Double-Angle', '3: Double-Angle with Defect'], 'LVDT 3', 60)

In [ ]:
plot_lines([load1[0:ind1], load2[0:ind2], load3[0:ind3]],[LVDT6_bridge1[0:ind1], LVDT6_bridge2[0:ind2], LVDT6_bridge3[0:ind3]],
           ['Load (kN)','Displacement (m)'],['1: Single-Angle','2: Double-Angle', '3: Double-Angle with Defect'], 'LVDT 6', 60)

In [ ]:
plot_lines([load1[0:ind1], load2[0:ind2], load3[0:ind3]],[LVDT3_bridge1[0:ind1]+LVDT6_bridge1[0:ind1], LVDT3_bridge2[0:ind2]+LVDT6_bridge2[0:ind2], LVDT3_bridge3[0:ind3]+LVDT6_bridge3[0:ind3]],
           ['Load (kN)','Displacement (m)'],['1: Single-Angle','2: Double-Angle', '3: Double-Angle with Defect'], 'Member DC Expansion', 60)

<font color='red'>Question: </font> How does the behavior of Member DC compare to that of Member AB? What do you expect and what is the actual behavior?

<font color='red'>Answer: </font>

# (3) Rotation at the top of the bridge

Let's explore the tiltmeter data. Figure 5 shows the location of the WSN tilt motes and the orientation of the two orthogonal axes. The direction of angle is given for both longitudinal direction (L) and transverse direction (T).

We will focus making a comparison between bridge 1 (single-angle) and bridge 2 (double-angle) for this section.

<center>
<img src="https://github.com/UCB-CE170a/Fall2021/raw/master/Homeworks/HW2/images/tilt_defination.jpg" alt="Node numbering" width="600" />
    
    Figure 5. The locations of WSN tilt motes.
</center>

In [ ]:
tilt1A = pd.read_csv('tilt_sync_1A.csv')
tilt1B = pd.read_csv('tilt_sync_1B.csv')
tilt2A = pd.read_csv('tilt_sync_2A.csv')
tilt2B = pd.read_csv('tilt_sync_2B.csv')

In [ ]:
# zero-out the tilt measurements
tilt1A['A'] = tilt1A['A'] - tilt1A['A'].loc[0]
tilt1B['B'] = tilt1B['B'] - tilt1B['B'].loc[0]
tilt2A['A'] = tilt2A['A'] - tilt2A['A'].loc[0]
tilt2B['B'] = tilt2B['B'] - tilt2B['B'].loc[0]

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.plot(tilt1A['Load(lb)']*lbs_to_kN, tilt1A['A'], label='Transverse')
ax.plot(tilt1B['Load(lb)']*lbs_to_kN, tilt1B['B'], label='Longitudinal')

plt.ylabel('Rotation (degrees)', fontsize = 10)
plt.xlabel('Load (kN)', fontsize = 10)
# plt.ylim([-3,7])
ax.legend()
ax.grid(color = [0.8, 0.8, 0.8], which = 'major')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.tick_params(direction="in", which = 'major', length = 6, width = 1)
ax.tick_params(direction="in", which = 'minor', length = 3, width = 0.6)
ax.set_title('Bridge 1: Single-Angle', fontsize = 14)

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.plot(tilt2A['Load(lb)']*lbs_to_kN, tilt2A['A'], label='Transverse')
ax.plot(tilt2B['Load(lb)']*lbs_to_kN, tilt2B['B'], label='Longitudinal')

plt.ylabel('Rotation (degrees)', fontsize = 10)
plt.xlabel('Load (kN)', fontsize = 10)
plt.ylim([-0.3, 0.6])
ax.legend()
ax.grid(color = [0.8, 0.8, 0.8], which = 'major')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.tick_params(direction="in", which = 'major', length = 6, width = 1)
ax.tick_params(direction="in", which = 'minor', length = 3, width = 0.6)
ax.set_title('Bridge 2: Double-Angle', fontsize = 14)

<font color='red'>Question: </font> Using the two rotation plots above, discuss how the two bridges behaved as they were about to fail. Do the failure patterns match the your observations?

<font color='red'>Answer: </font>

<font color='red'>Question: </font> Think about what role tiltmeters could play in providing early warning of model bridge failure.

<font color='red'>Answer: </font>

# (4) Behavior of individual members

Finally, let's explore the most interesting data, the distributed strain data collected from the fiber optics sensors. The model bridge memberes are divided into 4 planes: Bottom, Left-diagonal, Right-diagonal, and Middle plane. See Figure 7. 


<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw5/figures/figure1.png" alt="Node numbering" width="800" />
    
    Figure 7. Node numbering for the model bridge
</center>

As discussed earlier, the raw data collected by the fiber optics sensors are spatially distributed. Specifically, the data contains spatial strain information at each sample time for each fiber. The general preprocessing process is listed as the following:
* Cleaning: Convert data format, remove headers, convert units, remove meaningless items... 
* Time synchronization: all the collected data is time-stamped. Unfortunately, the timing systems for different measurement equipment might be slightly different. For instance, the clock in the fiber optics analyzer is 2 minutes and 19 seconds faster than the one measuring the loading process. Synchronizing times across all equipment make the comparison/plotting of different measurements possible. 
* Resampling: sometimes, the sampling rates across different equipment are different. To compare different measurements from different machines (i.e., load vs. strain), one needs to resample the collected data to match the time stamp.
* Normalization: some fibers are pre tensioned, meaning the starting measurement may not be 0. Doing normalization for the data makes the measurement (strain) comparable across different bridges. 
 
<font color='red'>Note: the data you see in this section is far from perfect (the fibers are attached by you!). If some data does not make any sense to you, think about the potential causes for that. Try your best to discuss your findings.  </font>

For this section of the assignment, we'll focus on a few members from the <b><i>second bridge (double-angle). </b></i>

We first reread the load data, which has now been synchronized with the fiber optic data in time:

In [ ]:
conv_load  = pd.read_csv('conv_load.csv')
gauge_length = 0.065

In [ ]:
sample_data = pd.read_csv('dict_CJ.csv')
sample_data.head(5)

The first row of the data provides the spatial location for each strain measurement points alone the fiber. The rest of the rows are the strain measurements across the fiber (0.065 m resolution) for every sample time. To access the strain measurement for certain truss members, one need to know the corresponding spatial locations of the member on the fiber. Luckily, our kind GSI did indexing (truss member end points to fiber locations) for all the truss members for all three bridges, so we know how to access the member from the data.

## Time (load) Data Analysis
Since different groups wired fiber cables in different ways and analyzing both spatial and temporal data requires skills beyond the boundary of this class, simplified strain data is used in the later part of the assignment. Certain truss members are extracted from the data for all bridges, and only the average (spatial averaged) strain for each truss member is provided along with the corresponding load levels.

## (4.1) Diagonal Members

In [ ]:
sensor_AG = pd.read_csv('comp_AG_timeseries.txt').to_numpy()
sensor_BI = pd.read_csv('comp_BI_timeseries.txt').to_numpy()
sensor_DH = pd.read_csv('comp_DH_timeseries.txt').to_numpy()
sensor_CJ = pd.read_csv('comp_CJ_timeseries.txt').to_numpy()

load = conv_load['load'].to_numpy()

In [ ]:
ax = plt.axes()

ax.plot(conv_load['load']*lbs_to_kN,sensor_AG, label='AG')
ax.plot(conv_load['load']*lbs_to_kN,sensor_BI, label='BI')
ax.plot(conv_load['load']*lbs_to_kN,sensor_DH, label='DH')
ax.plot(conv_load['load']*lbs_to_kN,sensor_CJ, label='CJ')

plt.ylabel('Strain (micro strain)', fontsize = 10)
plt.xlabel('Load (kN)', fontsize = 10)
ax.legend()
ax.set_title('Diagonal Members', fontsize = 14)

<font color='red'>Question: </font> Describe any similarities and differences between the members. What is the strain at approximately 10 kN? Does it match your predictions? Explain your reasoning.

<font color='red'>Answer: </font>

## (4.2) Bottom Members

In [ ]:
sensor_AM = pd.read_csv('tens_AM_timeseries.txt').to_numpy()
sensor_MB = pd.read_csv('tens_MB_timeseries.txt').to_numpy()
sensor_DN = pd.read_csv('tens_DN_timeseries.txt').to_numpy()
sensor_NC = pd.read_csv('tens_NC_timeseries.txt').to_numpy()

In [ ]:
ax = plt.axes()

plt.plot(conv_load['load']*lbs_to_kN,sensor_AM, label='AM')
plt.plot(conv_load['load']*lbs_to_kN,sensor_MB, label='MB')
plt.plot(conv_load['load']*lbs_to_kN,sensor_DN, label='DN')
plt.plot(conv_load['load']*lbs_to_kN,sensor_NC, label='NC')

plt.ylabel('Strain (micro strain)', fontsize = 10)
plt.xlabel('Load (kN)', fontsize = 10)
ax.legend()
ax.set_title('Bottom Members', fontsize = 14)

<font color='red'>Question: </font> Describe any similarities and differences between the members. What is the strain at approximately 10 kN? Does it match your predictions? Explain your reasoning.

<font color='red'>Answer: </font>

## (4.3) Middle Section Diagonal Members

In [ ]:
sensor_AH = pd.read_csv('AH_timeseries.txt').to_numpy()
sensor_JE = pd.read_csv('JE_timeseries.txt').to_numpy()
sensor_GF = pd.read_csv('GF_timeseries.txt').to_numpy()
sensor_CI = pd.read_csv('CI_timeseries.txt').to_numpy()

In [ ]:
ax = plt.axes()

plt.plot(conv_load['load']*lbs_to_kN,sensor_AH, label='AH')
plt.plot(conv_load['load']*lbs_to_kN,sensor_JE, label='JE')
plt.plot(conv_load['load']*lbs_to_kN,sensor_GF, label='GF')
plt.plot(conv_load['load']*lbs_to_kN,sensor_CI, label='CI')

plt.ylabel('Strain (micro strain)', fontsize = 10)
plt.xlabel('Load (kN)', fontsize = 10)
ax.legend()
ax.set_title('Middle Section Diagonal Members', fontsize = 14)

<font color='red'>Question: </font> Describe any similarities and differences between the members. Is this plot as you would expect? Explain your reasoning.

<font color='red'>Answer: </font>

# Discussion

<font color='red'><b>Question (1): </b></font> Use the strain-load plots for the diagonal members (AG, BI, DH, JC) from problem (4.1) to revise the stiffness of the material in your FE model. Are your estimations the same across the four different members? How do they compare to the theoretical value?

* Hint1: "Load" from data is the system (total) load. Go back to Quiz 2 to find the equation to convert total load to the load felt by each member. Again, pay attention to number of angle bars for each member.

* Hint2: Reading the values from the graph is acceptable.

<font color='red'>Answer: </font>

<font color='red'><b>Question (2):</b> </font> Think back to what you observed during the tests. (Watch the videos <a href="https://drive.google.com/drive/folders/1HdlADyA2HnBFveScQDFVCDR5zo17PDfv?usp=sharing">here</a> if it helps.) Describe the deformation mechanisms of the bridges as a whole at different loading stages. 

<font color='red'>Answer: </font>

<font color='red'><b>Question (3):</b></font> Discuss the validity of the assumptions that we made in the analysis given in Quiz 2.


<font color='red'>Answer: </font>

<font color='red'><b>Question (4):</b></font> If you could improve the simulation analysis, what would you do?


<font color='red'>Answer: </font>

<font color='red'><b>Question (5):</b></font> If you were allowed to add more sensors to the model bridge, what and where would you add? 

<font color='red'>Answer: </font>